Cifrado Asimétrico — ECC (Criptografía de Curva Elíptica)

In [ ]:
!pip install cryptography

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF #Derivacion de claves seguras
from cryptography.hazmat.primitives.ciphers.aead import AESGCM #Cifrado simetrico
import os, base64

# Generar par de claves ECC
private_key = ec.generate_private_key(ec.SECP256R1()) #SECP256R1 Como curva elíptica estándar
public_key = private_key.public_key()

# Simulamos el intercambio de clave pública
peer_private = ec.generate_private_key(ec.SECP256R1())
peer_public = peer_private.public_key()

# Derivar una clave compartida usando ECDH
shared_key = private_key.exchange(ec.ECDH(), peer_public) #ECDH Protocolo para clave secreta

# Derivar clave simétrica a partir de la compartida (usando HKDF)
derived_key = HKDF(             #Derivacion de claves seguras
    algorithm=hashes.SHA256(), #SHA-256 como función hash base
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key) #Genera una clave binaria de 32 bytes para cifrar

# Cifrar mensaje con AES-GCM (usando la clave derivada)
aesgcm = AESGCM(derived_key) #Cifrado autenticado en combinación de AES-GCM(Galios=codigos autenticos)
nonce = os.urandom(12)
plaintext = b"Universidad central del ecuador"
ciphertext = aesgcm.encrypt(nonce, plaintext, None)

print("Mensaje cifrado (base64):", base64.b64encode(ciphertext).decode())

# Descifrar
decrypted = aesgcm.decrypt(nonce, ciphertext, None) #nonce es el número único generado en el cifrado
print("Mensaje descifrado:", decrypted.decode())


Mensaje cifrado (base64): +44zp81VSeTlRQmbbG1AMuQIRstUm085jolQdZTGK0ADeD47WvxKtJzzJN+E6qQ=
Mensaje descifrado: Universidad central del ecuador
